In [1]:
import boto3

# 載入 .env 檔案中的環境變數
from dotenv import load_dotenv
import os
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response["Account"]

print(f"AWS Account ID: {account_id}")

AWS Account ID: 145745076948


In [2]:
# 建立 IAM 客戶端
iam_client = boto3.client("iam")

# 取得所有角色
def list_roles():
    try:
        # 透過客戶端取得所有角色
        response = iam_client.list_roles()

        # 儲存所有角色的名稱和 ARN
        roles = {role["RoleName"]: role["Arn"] for role in response["Roles"]}
        return roles
    except Exception as e:
        print(f"無法取得角色列表。錯誤訊息：{e}")
        return None


# 自訂義篩選角色的函數，函數需傳入關鍵字進行篩選
def get_role_arn(role_keyword):
    try:
        # 呼叫 list_roles 方法以取得所有角色
        response = iam_client.list_roles()
        # 遍歷所有角色以篩選出包含指定關鍵字的角色 ARN
        for role in response["Roles"]:
            if role_keyword in role["RoleName"]:
                return role["Arn"]
        print(f"無法找到包含關鍵字 '{role_keyword}' 的角色。")
        return None
    except Exception as e:
        print(f"無法取得角色列表。錯誤訊息：{e}")
        return None


# 定義要搜尋的角色關鍵字
database_role_keyword = "ComprehendDataAccessRole"
translate_role_keyword = "TranslateDemoRole"

# 取得包含 'ComprehendDataAccessRole' 的角色 ARN
database_access_role_arn = get_role_arn(
    database_role_keyword
)
# 取得包含 'TranslateDemoRole' 的角色 ARN
translate_access_role_arn = get_role_arn(
    translate_role_keyword
)

# 輸出查看
print(
    f"Database Access Role ARN：【{database_access_role_arn}】"
)
print(
    f"Translate Access Role ARN：【{translate_access_role_arn}】"
)

Database Access Role ARN：【arn:aws:iam::145745076948:role/service-role/c133864a3391506l8170626t1w-ComprehendDataAccessRole-GwNd2eO8idwN】
Translate Access Role ARN：【arn:aws:iam::145745076948:role/c133864a3391506l8170626t1w1457450-TranslateDemoRole-ZE40xTBQENE0】


In [3]:
# 建立 S3 客戶端
s3_client = boto3.client("s3")

# 列出 Buckets
def list_buckets():
    try:
        # 呼叫 list_buckets 方法以取得所有 bucket
        response = s3_client.list_buckets()

        # 從回應中提取每個 bucket 名稱並篩選包含 'labbucket' 的
        bucket_names = [
            bucket["Name"]
            for bucket in response["Buckets"]
            if "labbucket" in bucket["Name"]
        ]
        return bucket_names
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


# 取得篩選後的 bucket 名稱
bucket_names = list_buckets()

# 顯示取得的 bucket 名稱
if bucket_names:
    print("包含 'labbucket' 的 Bucket 名稱列表：")
    for name in bucket_names:
        print(name)
else:
    print("沒有找到包含 'labbucket' 的 Bucket。")

包含 'labbucket' 的 Bucket 名稱列表：
c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk


In [4]:
# 生成全域唯一識別碼
import uuid

# 處理 JSON 格式的數據
import json

# 從 time 模組匯入 sleep 函數，用於讓程式暫停指定的秒數
from time import sleep

# 自訂義篩選 Bucket 的函數，需傳入關鍵字進行篩選
def list_buckets_with_keyword(keyword):
    try:
        # 調用 list_buckets 取得所有 bucket
        response = s3_client.list_buckets()
        bucket_names = [
            # 篩選包含特定關鍵字的名稱
            # 特別注意這是 inline 的表達式，斷行不要縮排
            bucket["Name"]
            for bucket in response["Buckets"]
            if keyword in bucket["Name"]
        ]
        if bucket_names:
            # 假設只取第一個符合條件的 bucket 名稱
            return bucket_names[0]
        else:
            print(f"沒有找到包含 '{keyword}' 的 bucket。")
            return None
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


# 取得包含 'labbucket' 的 S3 bucket 名稱
bucket = list_buckets_with_keyword("labbucket")

# 輸出以上步驟取得的 ARN
print(f"S3 Bucket 名稱：【{bucket}】")

S3 Bucket 名稱：【c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk】


In [5]:
media_input_uri = f"s3://{bucket}/lab71/transcribe-sample/test.wav"

In [6]:
# 建立 AWS Transcribe 的客戶端
transcribe_client = boto3.client("transcribe")

# 生成 UUID 作為此轉錄作業的名稱
job_uuid = uuid.uuid1()
# 組合轉錄作業名稱
transcribe_job_name = f"transcribe-job-{job_uuid}"
# 自訂義轉錄輸出的檔案名稱
transcribe_output_filename = "transcribe_output.txt"

# 啟動轉錄作業
response = transcribe_client.start_transcription_job(
    # 轉錄作業的名稱
    TranscriptionJobName=transcribe_job_name,
    # 轉錄的媒體檔案 URI
    Media={"MediaFileUri": media_input_uri},
    MediaFormat="wav",
    # 音訊中的語言代碼
    LanguageCode="en-US",
    # 轉錄結果的輸出 S3 bucket
    OutputBucketName=bucket,
    # 輸出結果檔案的名稱
    OutputKey=transcribe_output_filename,
)

In [7]:
response

{'TranscriptionJob': {'TranscriptionJobName': 'transcribe-job-74bafd16-968f-11ef-ad51-3a328b8844c7',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'en-US',
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 's3://c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/lab71/transcribe-sample/test.wav'},
  'StartTime': datetime.datetime(2024, 10, 30, 15, 20, 36, 218000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 10, 30, 15, 20, 36, 198000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '98880c40-20fb-4953-ab60-09b6bd08f6a3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '98880c40-20fb-4953-ab60-09b6bd08f6a3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '369',
   'date': 'Wed, 30 Oct 2024 07:20:35 GMT'},
  'RetryAttempts': 0}}

In [8]:
job = None
while True:
    # 指定轉錄工作狀態，並將結果存放在變數 job 中
    # transcribe_job_name 是轉錄工作的名稱。
    job = transcribe_client.get_transcription_job(
        TranscriptionJobName=transcribe_job_name
    )

    # 檢查轉錄工作狀態
    if job["TranscriptionJob"]["TranscriptionJobStatus"] in ["COMPLETED", "FAILED"]:
        # 如果符合條件，則跳出循環
        break
    print(".", end="")
    sleep(20)

print(job["TranscriptionJob"]["TranscriptionJobStatus"])

.COMPLETED


In [9]:
transcription_file = job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
print(transcription_file)

https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/transcribe_output.txt


In [10]:
job

{'TranscriptionJob': {'TranscriptionJobName': 'transcribe-job-74bafd16-968f-11ef-ad51-3a328b8844c7',
  'TranscriptionJobStatus': 'COMPLETED',
  'LanguageCode': 'en-US',
  'MediaSampleRateHertz': 44100,
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 's3://c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/lab71/transcribe-sample/test.wav'},
  'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/transcribe_output.txt'},
  'StartTime': datetime.datetime(2024, 10, 30, 15, 20, 36, 218000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 10, 30, 15, 20, 36, 198000, tzinfo=tzlocal()),
  'CompletionTime': datetime.datetime(2024, 10, 30, 15, 20, 44, 535000, tzinfo=tzlocal()),
  'Settings': {'ChannelIdentification': False, 'ShowAlternatives': False}},
 'ResponseMetadata': {'RequestId': 'e7a30535-8e15-408f-b19f-76b3ae86f256',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '

In [11]:
job["TranscriptionJob"]["TranscriptionJobStatus"]

'COMPLETED'

In [12]:
job["TranscriptionJob"]["Transcript"]

{'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/transcribe_output.txt'}

In [13]:
with open(transcribe_output_filename, "wb") as f:
    s3_client.download_fileobj(
        bucket,
        transcribe_output_filename,
        f
    )

In [14]:
with open(transcribe_output_filename) as f:
    data = json.load(f)

In [15]:
data

{'jobName': 'transcribe-job-74bafd16-968f-11ef-ad51-3a328b8844c7',
 'accountId': '145745076948',
 'status': 'COMPLETED',
 'results': {'transcripts': [{'transcript': 'Test. Hello. Hello. Hello. This is a test test test test.'}],
  'items': [{'id': 0,
    'type': 'pronunciation',
    'alternatives': [{'confidence': '0.996', 'content': 'Test'}],
    'start_time': '0.009',
    'end_time': '0.56'},
   {'id': 1,
    'type': 'punctuation',
    'alternatives': [{'confidence': '0.0', 'content': '.'}]},
   {'id': 2,
    'type': 'pronunciation',
    'alternatives': [{'confidence': '0.997', 'content': 'Hello'}],
    'start_time': '0.66',
    'end_time': '1.19'},
   {'id': 3,
    'type': 'punctuation',
    'alternatives': [{'confidence': '0.0', 'content': '.'}]},
   {'id': 4,
    'type': 'pronunciation',
    'alternatives': [{'confidence': '0.996', 'content': 'Hello'}],
    'start_time': '1.2',
    'end_time': '1.7'},
   {'id': 5,
    'type': 'punctuation',
    'alternatives': [{'confidence': '0.0'

In [16]:
# 生成全域唯一識別碼
import uuid

# 建立 AWS Translate 的客戶端，用於呼叫 AWS Translate 服務
translate_client = boto3.client(service_name="translate")

# 定義輸入數據的 S3 路徑
input_data = f"s3://{bucket}/lab71/translate-sample"

# 定義轉譯後輸出結果的 S3 路徑
output_data = f"s3://{bucket}"

# 生成一個 UUID，作為此次翻譯作業的唯一識別碼
job_uuid = uuid.uuid1()

# 將 UUID 加入到 "translate-job-" 的前綴中
translate_job_name = f"translate-job-{job_uuid}"

# 啟動文本翻譯作業
translate_job_submission = translate_client.start_text_translation_job(
    # 設定翻譯作業的名稱
    JobName=translate_job_name,
    # 配置輸入數據的參數，包含 S3 路徑和數據類型
    InputDataConfig={"S3Uri": input_data, "ContentType": "text/plain"},
    # 配置輸出數據的參數，指定輸出結果的 S3 路徑
    OutputDataConfig={"S3Uri": output_data},
    # 設定存取 AWS Translate 服務所需的 IAM 角色 ARN，以授權訪問 S3 資源
    DataAccessRoleArn=translate_access_role_arn,
    # 指定源語言的語言代碼，這裡為英文（en）
    SourceLanguageCode="en",
    # 指定目標語言的語言代碼，這裡為西班牙語（es）
    TargetLanguageCodes=["es"],
)

# 從作業提交的回應中提取 Job ID
# 將 ID 儲存到 translate_job_id 變數中
translate_job_id = translate_job_submission["JobId"]

# 使用 ID 來取得狀態，等待作業完成
while True:
    translate_job = translate_client.describe_text_translation_job(
        JobId=translate_job_id
    )
    if translate_job["TextTranslationJobProperties"]["JobStatus"] in [
        "COMPLETED",
        "FAILED",
    ]:
        break
    sleep(20)
    print(".", end="")

print(translate_job["TextTranslationJobProperties"]["JobStatus"])

.................................................COMPLETED


In [17]:
# 獲取當前 AWS 帳戶的 ID
account_id = boto3.client("sts").get_caller_identity().get("Account")

# 定義翻譯輸出結果的 S3 路徑
# 使用帳戶 ID、固定的 "TranslateText" 標識、翻譯作業的 ID，組成路徑名稱
translate_output_path = f"{account_id}-TranslateText-{translate_job_id}/"

In [18]:
translate_output_path

'145745076948-TranslateText-eb981df8b07d694c8b4b74985fc29483/'

In [19]:
# 建立 S3 資源對象
s3_resource = boto3.resource("s3")

# 使用指定名稱建立 bucket
my_bucket = s3_resource.Bucket(bucket)

# 過濾 bucket 中以 translate_output_path 作為前綴的物件
for my_bucket_object in my_bucket.objects.filter(
    Prefix=translate_output_path
):
    # 提取每個物件的 key，文件名稱及路徑
    file = my_bucket_object.key

    # 檢查文件是否以 "txt" 結尾，即確認文件格式為文本文件
    if file.endswith("txt"):
        # 去除路徑前綴 translate_output_path，使 file 僅包含文件名稱
        file = file.lstrip(translate_output_path)
        # 去除開頭的 "/" 符號，以獲得乾淨的文件名稱
        file = file.lstrip("/")
        
        # 輸出文件名稱，便於查看下載的文件
        print(file)
        
        # 打開本地文件，準備以二進制寫入方式將文件下載到本地
        with open(file, "wb") as f:
            # 使用 s3_client 將 S3 中的物件下載到本地文件
            s3_client.download_fileobj(bucket, my_bucket_object.key, f)


.test.txt


In [20]:
# 呼叫 Polly 語音合成服務
polly_client = boto3.client("polly")

# 指定要讀取的文件路徑
itemname = "lab71/polly-sample/es.test.txt"

# 取得 bucket 中的特定文件對象
obj = s3_resource.Object(bucket, itemname)

# 取得文件內容，並將二進制數據解碼為 `UTF-8` 編碼的字串
body = obj.get()["Body"].read().decode("utf-8")

# 啟動語音合成作業
response = polly_client.start_speech_synthesis_task(
    # 指定使用標準語音合成引擎
    Engine="standard",
    # 指定輸出音訊格式
    OutputFormat="mp3",
    # 將輸出結果儲存到的 S3 bucket
    OutputS3BucketName=bucket,
    # 傳入要合成語音的文本內容
    Text=body,
    # 指定語音 ID，這裡使用西班牙語的 Lucia 聲音
    VoiceId="Lucia"
)

In [21]:
task_id = response["SynthesisTask"]["TaskId"]
print(task_id)

fcb91546-20b5-4f1c-bd61-aaf9643c5e0d


In [22]:
while True:
    # 查詢語音合成作業的狀態
    polly_job = polly_client.get_speech_synthesis_task(TaskId=task_id)
    
    # 檢查作業的狀態是否為 `completed` 或 `failed``
    if polly_job["SynthesisTask"]["TaskStatus"] in ["completed", "failed"]:
        break
    
    # 若作業仍在進行中，暫停 20 秒以避免頻繁查詢狀態
    sleep(20)
    # 每20秒檢查一次，假如仍在進行，就會看到輸出一個點 `.`
    print(".", end="")

# 循環結束後會輸出最終的作業狀態 `completed` 或 `failed``
print(polly_job["SynthesisTask"]["TaskStatus"])

.completed


In [23]:
# 建立 S3 客戶端
s3_client = boto3.client("s3")

# 定義要下載的文件名稱
polly_output_filename = f"{task_id}.mp3"

# 以二進制寫入模式 ("wb") 打開輸出文件
with open(polly_output_filename, "wb") as f:
    # 將 S3 中指定的 的文件下載至本地
    s3_client.download_fileobj(bucket, polly_output_filename, f)

# 原路返回

確認名稱

In [ ]:
bucket = list_buckets_with_keyword("labbucket")
bucket

完成上傳並確認音頻文件成功上傳

In [ ]:
import boto3

# 上傳本地音頻文件至 S3 的指定路徑
with open(polly_output_filename, "rb") as f:
    s3_client.upload_fileobj(
        f,
        bucket,
        f"lab71/transcribe-sample/{polly_output_filename}"
    )

# 驗證上傳成功後，開始設定轉錄作業
transcribe_client = boto3.client("transcribe")
media_input_uri = f"s3://{bucket}/lab71/transcribe-sample/{polly_output_filename}"

# 建立 UUID 作為轉錄作業名稱
job_uuid = uuid.uuid1()
transcribe_job_name = f"transcribe-job-{job_uuid}"

# 啟動轉錄作業
response = transcribe_client.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    Media={"MediaFileUri": media_input_uri},
    MediaFormat="mp3",  # 確保格式與文件一致
    LanguageCode="es-ES",
    OutputBucketName=bucket,
    OutputKey="transcribe_output_es.txt"
)

# 等待轉錄作業完成
while True:
    job = transcribe_client.get_transcription_job(TranscriptionJobName=transcribe_job_name)
    if job["TranscriptionJob"]["TranscriptionJobStatus"] in ["COMPLETED", "FAILED"]:
        break
    sleep(20)
    print(".", end="")

# 確認作業完成並顯示結果 URI 或失敗原因
if job["TranscriptionJob"]["TranscriptionJobStatus"] == "COMPLETED":
    transcription_file_uri = job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
    print("轉錄文件 URI:", transcription_file_uri)
else:
    print("轉錄作業失敗，狀態:", job["TranscriptionJob"]["TranscriptionJobStatus"])
    if "FailureReason" in job["TranscriptionJob"]:
        print("失敗原因:", job["TranscriptionJob"]["FailureReason"])


..轉錄文件 URI: https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/transcribe_output_es.txt


將西班牙語音頻文件轉錄成文本

In [ ]:
# 下載並檢查轉錄文件
transcribe_output_filename = "transcribe_output_es.txt"

# 使用 S3 客戶端從 S3 中下載轉錄結果文件
with open(transcribe_output_filename, "wb") as f:
    s3_client.download_fileobj(bucket, transcribe_output_filename, f)

# 查看轉錄內容
with open(transcribe_output_filename, "r") as f:
    spanish_text = f.read()
print("轉錄的西班牙語文本:", spanish_text)

轉錄的西班牙語文本: {"jobName":"transcribe-job-8f087450-96a3-11ef-ad51-3a328b8844c7","accountId":"145745076948","status":"COMPLETED","results":{"transcripts":[{"transcript":"prueba de prueba. Esta es una prueba."}],"items":[{"id":0,"type":"pronunciation","alternatives":[{"confidence":"0.999","content":"prueba"}],"start_time":"0.009","end_time":"0.349"},{"id":1,"type":"pronunciation","alternatives":[{"confidence":"0.999","content":"de"}],"start_time":"0.36","end_time":"0.46"},{"id":2,"type":"pronunciation","alternatives":[{"confidence":"0.999","content":"prueba"}],"start_time":"0.469","end_time":"0.819"},{"id":3,"type":"punctuation","alternatives":[{"confidence":"0.0","content":"."}]},{"id":4,"type":"pronunciation","alternatives":[{"confidence":"0.592","content":"Esta"}],"start_time":"0.829","end_time":"1.08"},{"id":5,"type":"pronunciation","alternatives":[{"confidence":"0.998","content":"es"}],"start_time":"1.09","end_time":"1.24"},{"id":6,"type":"pronunciation","alternatives":[{"confidence":"0

In [76]:
# 提取西班牙語文本
spanish_text = transcribe_data["results"]["transcripts"][0]["transcript"]
print("轉錄的西班牙語文本:", spanish_text)

轉錄的西班牙語文本: prueba de prueba. Esta es una prueba.


將西班牙語文本翻譯為英文

In [77]:
# 使用 Amazon Translate 將西班牙語文本翻譯為英文
response = translate_client.translate_text(
    Text=spanish_text,
    SourceLanguageCode="es",
    TargetLanguageCode="en"
)

# 獲取翻譯後的英文文本
english_text = response['TranslatedText']
print("翻譯後的英文文本:", english_text)


翻譯後的英文文本: test test. This is a test.


將翻譯的英文文本轉換成語音

In [78]:
# 啟動語音合成作業
response = polly_client.start_speech_synthesis_task(
    Engine="standard",
    OutputFormat="mp3",
    OutputS3BucketName=bucket,
    Text=english_text,
    VoiceId="Joanna"  # 美式英文的 Joanna 聲音
)

# 等待語音合成作業完成並獲取文件 URI
task_id = response["SynthesisTask"]["TaskId"]
while True:
    polly_job = polly_client.get_speech_synthesis_task(TaskId=task_id)
    if polly_job["SynthesisTask"]["TaskStatus"] in ["completed", "failed"]:
        break
    sleep(20)
    print(".", end="")

# 確認合成成功並獲取文件 URI
if polly_job["SynthesisTask"]["TaskStatus"] == "completed":
    audio_file_uri = polly_job["SynthesisTask"]["OutputUri"]
    print("英文語音文件 URI:", audio_file_uri)
else:
    print("語音合成失敗:", polly_job["SynthesisTask"]["TaskStatus"])


.英文語音文件 URI: https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/e76b99e4-a8c2-4f40-b5f0-92113911925f.mp3


使用 boto3 下載該文件

In [79]:
print("Bucket 名稱:", bucket)
print("S3 文件完整路徑:", s3_file_key)

Bucket 名稱: c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk
S3 文件完整路徑: 91f81a74-3ca3-48c8-a731-309bdff0b968.mp3


In [86]:
# 根據 S3 文件的實際路徑修正文件 key
s3_file_key = f"{task_id}.mp3"  # 文件直接位於根目錄

# 定義本地文件名稱
local_filename = "downloaded_audio.mp3"

# 檢查文件是否存在於 S3 中並下載
try:
    # 確認 bucket 名稱和 key 是否正確
    print("Bucket 名稱:", bucket)
    print("S3 文件完整路徑:", s3_file_key)
    
    # 檢查文件是否存在
    s3_client.head_object(Bucket=bucket, Key=s3_file_key)
    print("文件存在於 S3 中，可以繼續進行下載。")

    # 下載文件
    with open(local_filename, "wb") as f:
        s3_client.download_fileobj(bucket, s3_file_key, f)

    print(f"英文語音文件已成功下載到本地：{local_filename}")

except s3_client.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == '404':
        print(
            f"文件 '{s3_file_key}' 在 bucket '{bucket}' 中不存在。"
            "請檢查路徑和文件名是否正確。"
        )
    else:
        print("其他 S3 錯誤:", e)
except Exception as ex:
    print("下載過程中發生其他錯誤:", ex)


Bucket 名稱: c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk
S3 文件完整路徑: e76b99e4-a8c2-4f40-b5f0-92113911925f.mp3
文件存在於 S3 中，可以繼續進行下載。
英文語音文件已成功下載到本地：downloaded_audio.mp3


從西班牙語音檔進行轉錄，將 S3 中的西班牙語音檔轉換為西班牙文本

In [ ]:
# 設定轉錄作業名稱和音頻文件的 URI
job_uuid = uuid.uuid1()
transcribe_job_name = f"transcribe-job-{job_uuid}"
# 指向 S3 中的西班牙語音檔
media_input_uri = f"s3://{bucket}/{output_path}/{mp3_filename}"  

# 啟動轉錄作業
response = transcribe_client.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    Media={"MediaFileUri": media_input_uri},
    MediaFormat="mp3",
    # 設定語言為西班牙語
    LanguageCode="es-ES",
    OutputBucketName=bucket,
    OutputKey="transcribe_output_es.txt"
)

# 等待轉錄作業完成
while True:
    job = transcribe_client.get_transcription_job(TranscriptionJobName=transcribe_job_name)
    if job["TranscriptionJob"]["TranscriptionJobStatus"] in ["COMPLETED", "FAILED"]:
        break
    sleep(20)
    print(".", end="")

# 確認作業完成並顯示結果 URI 或失敗原因
if job["TranscriptionJob"]["TranscriptionJobStatus"] == "COMPLETED":
    transcription_file_uri = job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
    print("西班牙語轉錄文件 URI:", transcription_file_uri)
else:
    print("轉錄作業失敗，狀態:", job["TranscriptionJob"]["TranscriptionJobStatus"])
    if "FailureReason" in job["TranscriptionJob"]:
        print("失敗原因:", job["TranscriptionJob"]["FailureReason"])


..西班牙語轉錄文件 URI: https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/transcribe_output_es.txt


轉錄完成後，下載生成的西班牙語文本，然後進行反向翻譯

In [53]:
# 定義下載後的文件名稱
transcribe_output_filename = "transcribe_output_es.txt"

# 下載文件
with open(transcribe_output_filename, "wb") as f:
    s3_client.download_fileobj(bucket, "transcribe_output_es.txt", f)

print(f"轉錄的西班牙語文本已成功下載到本地：{transcribe_output_filename}")


轉錄的西班牙語文本已成功下載到本地：transcribe_output_es.txt


將西班牙文本翻譯回英文

In [54]:
import json

# 加載轉錄文本
with open(transcribe_output_filename, "r", encoding="utf-8") as f:
    transcribed_text = json.load(f)["results"]["transcripts"][0]["transcript"]

# 建立 Amazon Translate 客戶端
translate_client = boto3.client("translate")

# 執行翻譯
result = translate_client.translate_text(
    Text=transcribed_text,
    SourceLanguageCode="es",  # 西班牙語
    TargetLanguageCode="en"   # 英文
)

translated_text = result["TranslatedText"]
print("翻譯後的英文文本:", translated_text)


翻譯後的英文文本: test test. This is a test.


將英文文本轉換為英文語音

In [68]:
# 執行語音合成
response = polly_client.start_speech_synthesis_task(
    Engine="standard",
    OutputFormat="mp3",
    OutputS3BucketName=bucket,
    Text=translated_text,
    # 英文語音，可以選擇其他英文語音
    VoiceId="Joanna"
)

# 取得語音合成任務的輸出 URI
polly_job_id = response["SynthesisTask"]["TaskId"]
polly_output_uri = response["SynthesisTask"]["OutputUri"]
print("合成的英文語音文件 URI:", polly_output_uri)


合成的英文語音文件 URI: https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-mhvzok7povxk/91f81a74-3ca3-48c8-a731-309bdff0b968.mp3


下載

In [70]:
# 從 polly_output_uri 中提取 bucket 名稱和 key
bucket_name = polly_output_uri.split('/')[3]
s3_file_key = '/'.join(polly_output_uri.split('/')[4:])

# 建立 S3 客戶端
s3_client = boto3.client("s3")

# 本地文件名稱
local_filename = f"{polly_job_id}.mp3"

# 從 S3 下載文件
with open(local_filename, "wb") as f:
    s3_client.download_fileobj(bucket_name, s3_file_key, f)

print(f"合成的英文語音文件已成功下載到本地：{local_filename}")

合成的英文語音文件已成功下載到本地：91f81a74-3ca3-48c8-a731-309bdff0b968.mp3
